In [1]:
import numpy as np
from pygsti.baseobjs import Basis

np.set_printoptions(precision=1, linewidth=1000)

c:\Users\jkskolf\AppData\Local\miniconda3\envs\qcvv\lib\site-packages\pygsti\baseobjs\opcalc\__init__.py:31: UserWarning: 
An optimized Cython-based implementation of `pygsti.baseobjs.opcalc` is available as
an extension, but couldn't be imported. This might happen if the
extension has not been built. `pip install cython`, then reinstall
pyGSTi to build Cython extensions. Alternatively, setting the
environment variable `PYGSTI_NO_CYTHON_WARNING` will suppress this
message.

  _warnings.warn(warn_msg)
c:\Users\jkskolf\AppData\Local\miniconda3\envs\qcvv\lib\site-packages\pygsti\circuits\circuitparser\__init__.py:60: UserWarning: 
An optimized Cython-based implementation of `pygsti.circuits.circuitparser` is available as
an extension, but couldn't be imported. This might happen if the
extension has not been built. `pip install cython`, then reinstall
pyGSTi to build Cython extensions. Alternatively, setting the
environment variable `PYGSTI_NO_CYTHON_WARNING` will suppress this
message.

 

In [2]:
# 1-qubit base case
pp = Basis.cast('PP', dim=4)
pauliNames = ["I", "X", "Y", "Z"]
initialStates = ["X+", "X-", "Y+","Y-", "Z+", "Z-"]


In [3]:
# Commutator Helper Functions
def commute(mat1, mat2):
    return mat1@mat2 + mat2@mat1

def anti_commute(mat1, mat2):
    return mat1@mat2 - mat2@mat1

In [4]:
# Hamiltonian Error Generator
def hamiltonian_error_generator(initial_state, pauli_index, pauliDict):
    return -1j*pauliDict[pauli_index]@initial_state@pauliDict["I"] + 1j*pauliDict["I"]@initial_state@pauliDict[pauli_index]

# Stochastic Error Generator
def stochastic_error_generator(initial_state, pauli_index, pauliDict):
    return pauliDict[pauli_index]@initial_state@pauliDict[pauli_index] - pauliDict["I"]@initial_state@pauliDict["I"]

# Pauli-correlation Error Generator
def pauli_correlation_error_generator(initial_state, pauli_index_1, pauli_index_2, pauliDict):
    return pauliDict[pauli_index_1]@initial_state@pauliDict[pauli_index_2] + pauliDict[pauli_index_2]@initial_state@pauliDict[pauli_index_1] - 0.5 * commute(commute(pauliDict[pauli_index_1], pauliDict[pauli_index_2]), initial_state)

# Anti-symmetric Error Generator
def anti_symmetric_error_generator(initial_state, pauli_index_1, pauli_index_2, pauliDict):
    return 1j*(pauliDict[pauli_index_1]@initial_state@pauliDict[pauli_index_2] - pauliDict[pauli_index_2]@initial_state@pauliDict[pauli_index_1] + 0.5 * commute(anti_commute(pauliDict[pauli_index_1], pauliDict[pauli_index_2]), initial_state))

In [5]:
# Convert basis; 1-q only at the moment
def convert_to_pauli(matrix):
    pauliNames = ["I", "X", "Y", "Z"]
    translationMatrix = pp.from_std_transform_matrix
    coefs = np.real_if_close(np.dot(translationMatrix,matrix.flatten()))
    return [(a,b) for (a,b) in zip(coefs,pauliNames) if abs(a) > 0.0001]

In [6]:
# Compute the set of measurable effects of Hamiltonian error generators operating on a single qubit in each of the specified eigenstates
hamiltonianIndices = ["X", "Y", "Z"]
hamiltonianErrorOutputs = dict()
for index in hamiltonianIndices:
    for state in initialStates:
        if state[-1] == "+":
            hamiltonianErrorOutputs[(index, state)] = 0.5*hamiltonian_error_generator(pp["I"], index, pp) + 0.5*hamiltonian_error_generator(pp[state[0]], index, pp)
        elif state[-1] == "-":
            hamiltonianErrorOutputs[(index, state)] = 0.5*hamiltonian_error_generator(pp["I"], index, pp) - 0.5*hamiltonian_error_generator(pp[state[0]], index, pp)

# Convert measurable effects into coefficients
for key in hamiltonianErrorOutputs:
    hamiltonianErrorOutputs[key] = convert_to_pauli(hamiltonianErrorOutputs[key])

for key in hamiltonianErrorOutputs:
    print(key, "\n", hamiltonianErrorOutputs[key])

('X', 'X+') 
 []
('X', 'X-') 
 []
('X', 'Y+') 
 [(1.0, 'Z')]
('X', 'Y-') 
 [(-1.0, 'Z')]
('X', 'Z+') 
 [(-1.0, 'Y')]
('X', 'Z-') 
 [(1.0, 'Y')]
('Y', 'X+') 
 [(-1.0, 'Z')]
('Y', 'X-') 
 [(1.0, 'Z')]
('Y', 'Y+') 
 []
('Y', 'Y-') 
 []
('Y', 'Z+') 
 [(1.0, 'X')]
('Y', 'Z-') 
 [(-1.0, 'X')]
('Z', 'X+') 
 [(1.0, 'Y')]
('Z', 'X-') 
 [(-1.0, 'Y')]
('Z', 'Y+') 
 [(-1.0, 'X')]
('Z', 'Y-') 
 [(1.0, 'X')]
('Z', 'Z+') 
 []
('Z', 'Z-') 
 []


In [7]:
# Compute the set of measurable effects of stochastic error generators operating on a single qubit in each of the specified eigenstates
stochasticIndices = ["X", "Y", "Z"]
stochasticErrorOutputs = dict()
for index in stochasticIndices:
    for state in initialStates:
        if state[-1] == "+":
            stochasticErrorOutputs[(index, state)] = 0.5*stochastic_error_generator(pp["I"], index, pp) + 0.5*stochastic_error_generator(pp[state[0]], index, pp)
        elif state[-1] == "-":
            stochasticErrorOutputs[(index, state)] = 0.5*stochastic_error_generator(pp["I"], index, pp) - 0.5*stochastic_error_generator(pp[state[0]], index, pp)

# Convert measurable effects into coefficients
for key in stochasticErrorOutputs:
    stochasticErrorOutputs[key] = convert_to_pauli(stochasticErrorOutputs[key])
for key in stochasticErrorOutputs:
    print(key, "\n", stochasticErrorOutputs[key])


('X', 'X+') 
 []
('X', 'X-') 
 []
('X', 'Y+') 
 [(-1.0, 'Y')]
('X', 'Y-') 
 [(1.0, 'Y')]
('X', 'Z+') 
 [(-1.0, 'Z')]
('X', 'Z-') 
 [(1.0, 'Z')]
('Y', 'X+') 
 [(-1.0, 'X')]
('Y', 'X-') 
 [(1.0, 'X')]
('Y', 'Y+') 
 []
('Y', 'Y-') 
 []
('Y', 'Z+') 
 [(-1.0, 'Z')]
('Y', 'Z-') 
 [(1.0, 'Z')]
('Z', 'X+') 
 [(-1.0, 'X')]
('Z', 'X-') 
 [(1.0, 'X')]
('Z', 'Y+') 
 [(-1.0, 'Y')]
('Z', 'Y-') 
 [(1.0, 'Y')]
('Z', 'Z+') 
 []
('Z', 'Z-') 
 []


In [8]:
# Compute the set of measurable effects of pauli-correlation error generators operating on a single qubit in each of the specified eigenstates
pauliCorrelationIndices = ["XY", "XZ", "YZ"]
pauliCorrelationErrorOutputs = dict()
for index in pauliCorrelationIndices:
    for state in initialStates:
        if state[-1] == "+":
            pauliCorrelationErrorOutputs[(index, state)] = 0.5*pauli_correlation_error_generator(pp["I"], index[0], index[1], pp) + 0.5*pauli_correlation_error_generator(pp[state[0]], index[0], index[1], pp)
        elif state[-1] == "-":
            pauliCorrelationErrorOutputs[(index, state)] = 0.5*pauli_correlation_error_generator(pp["I"], index[0], index[1], pp) - 0.5*pauli_correlation_error_generator(pp[state[0]], index[0], index[1], pp)

# Convert measurable effects into coefficients
for key in pauliCorrelationErrorOutputs:
    pauliCorrelationErrorOutputs[key] = convert_to_pauli(pauliCorrelationErrorOutputs[key])
for key in pauliCorrelationErrorOutputs:
    print(key, "\n", pauliCorrelationErrorOutputs[key])


('XY', 'X+') 
 [(1.0, 'Y')]
('XY', 'X-') 
 [(-1.0, 'Y')]
('XY', 'Y+') 
 [(1.0, 'X')]
('XY', 'Y-') 
 [(-1.0, 'X')]
('XY', 'Z+') 
 []
('XY', 'Z-') 
 []
('XZ', 'X+') 
 [(1.0, 'Z')]
('XZ', 'X-') 
 [(-1.0, 'Z')]
('XZ', 'Y+') 
 []
('XZ', 'Y-') 
 []
('XZ', 'Z+') 
 [(1.0, 'X')]
('XZ', 'Z-') 
 [(-1.0, 'X')]
('YZ', 'X+') 
 []
('YZ', 'X-') 
 []
('YZ', 'Y+') 
 [(1.0, 'Z')]
('YZ', 'Y-') 
 [(-1.0, 'Z')]
('YZ', 'Z+') 
 [(1.0, 'Y')]
('YZ', 'Z-') 
 [(-1.0, 'Y')]


In [9]:
# Compute the set of measurable effects of pauli-correlation error generators operating on a single qubit in each of the specified eigenstates
antiSymmetricIndices = ["XY", "XZ", "YZ"]
antiSymmetricErrorOutputs = dict()
for index in antiSymmetricIndices:
    for state in initialStates:
        if state[-1] == "+":
            antiSymmetricErrorOutputs[(index, state)] = 0.5*anti_symmetric_error_generator(pp["I"], index[0], index[1], pp) + 0.5*anti_symmetric_error_generator(pp[state[0]], index[0], index[1], pp)
        elif state[-1] == "-":
            antiSymmetricErrorOutputs[(index, state)] = 0.5*anti_symmetric_error_generator(pp["I"], index[0], index[1], pp) - 0.5*anti_symmetric_error_generator(pp[state[0]], index[0], index[1], pp)

# Convert measurable effects into coefficients
for key in antiSymmetricErrorOutputs:
    antiSymmetricErrorOutputs[key] = convert_to_pauli(antiSymmetricErrorOutputs[key])
for key in antiSymmetricErrorOutputs:
    print(key, "\n", antiSymmetricErrorOutputs[key])


('XY', 'X+') 
 [(-2.0, 'Z')]
('XY', 'X-') 
 [(-2.0, 'Z')]
('XY', 'Y+') 
 [(-2.0, 'Z')]
('XY', 'Y-') 
 [(-2.0, 'Z')]
('XY', 'Z+') 
 [(-2.0, 'Z')]
('XY', 'Z-') 
 [(-2.0, 'Z')]
('XZ', 'X+') 
 [(2.0, 'Y')]
('XZ', 'X-') 
 [(2.0, 'Y')]
('XZ', 'Y+') 
 [(2.0, 'Y')]
('XZ', 'Y-') 
 [(2.0, 'Y')]
('XZ', 'Z+') 
 [(2.0, 'Y')]
('XZ', 'Z-') 
 [(2.0, 'Y')]
('YZ', 'X+') 
 [(-2.0, 'X')]
('YZ', 'X-') 
 [(-2.0, 'X')]
('YZ', 'Y+') 
 [(-2.0, 'X')]
('YZ', 'Y-') 
 [(-2.0, 'X')]
('YZ', 'Z+') 
 [(-2.0, 'X')]
('YZ', 'Z-') 
 [(-2.0, 'X')]
